<a href="https://colab.research.google.com/github/rahiakela/advanced-natural-language-processing-with-tensorflow-2/blob/main/4-transfer-learning/1_understanding_sentiment_using_glove_based_transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Understanding Sentiment using GloVe based transfer learning

We have used BiLSTM model to predict the sentiment of IMDb movie reviews. That model learned embeddings of the words from scratch. This model had an accuracy of `83.55%` on the test set, while the SOTA result was closer to `97.4%`. If pre-trained embeddings are used, we expect an increase in model accuracy. 

Let's try this out and see the impact of transfer learning on this model.

##Setup

In [3]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.layers import Embedding, LSTM, Bidirectional, Dense, Dropout

import numpy as np
import pandas as pd

tf.__version__

'2.5.0'

In [2]:
######## GPU CONFIGS FOR RTX 2070 ###############
## Please ignore if not training on GPU       ##
## this is important for running CuDNN on GPU ##

tf.keras.backend.clear_session() #- for easy reset of notebook state

# chck if GPU can be seen by TF
tf.config.list_physical_devices('GPU')
# only if you want to see how commands are executed
#tf.debugging.set_log_device_placement(True)
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.experimental.set_memory_growth(gpus[0], True)
    tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)
###############################################

1 Physical GPUs, 1 Logical GPU


In [ ]:
# Download the GloVe embeddings
!wget -q http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip

##Loading IMDb training data

TensorFlow Datasets or the tfds package will be used to load the data:

In [ ]:
imdb_train, ds_info = tfds.load(name="imdb_reviews", split="train", with_info=True, as_supervised=True)
imdb_test = tfds.load(name="imdb_reviews", split="test", as_supervised=True)

In [5]:
# Check label and example from the dataset
for example, label in imdb_train.take(1):
  print(example, "\n", label)

tf.Tensor(b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.", shape=(), dtype=string) 
 tf.Tensor(0, shape=(), dtype=int64)


## Create Vocab and Encoder

After the training and test sets are loaded, the content of the reviews needs to be tokenized and encoded:

In [9]:
# Use the default tokenizer settings
tokenizer = tfds.deprecated.text.Tokenizer()

vocabulary_set = set()
MAX_TOKENS = 0

for example, label in imdb_train:
  some_tokens = tokenizer.tokenize(example.numpy())
  if MAX_TOKENS < len(some_tokens):
    MAX_TOKENS = len(some_tokens)
  vocabulary_set.update(some_tokens)

We tokenizes the review text and constructs a vocabulary.
This vocabulary is used to construct a tokenizer:

In [10]:
imdb_encoder = tfds.deprecated.text.TokenTextEncoder(vocabulary_set, lowercase=True, tokenizer=tokenizer)
vocab_size = imdb_encoder.vocab_size

print(vocab_size, MAX_TOKENS)

93931 2525


Note that text was converted to lowercase before encoding. Converting to lowercase helps reduce the vocabulary size and may benefit the lookup of corresponding GloVe vectors. Note that capitalization may contain important information, which may help in tasks such as NER.

In [11]:
# Lets verify tokenization and encoding works
for example, label in imdb_train.take(1):
  print(example, "\n")
  encoded = imdb_encoder.encode(example.numpy())
  print(encoded, "\n")
  print(imdb_encoder.decode(encoded))

tf.Tensor(b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.", shape=(), dtype=string) 

[89793, 90851, 41810, 64523, 42842, 46066, 68939, 53750, 64766, 77911, 56622, 65755, 81772, 46086, 74649, 85007, 75996, 50792, 86109, 92051, 63423, 81211, 89793, 59535, 59974, 64766, 83530, 58701, 88357, 56622, 72891, 63079, 83530, 92051, 65894, 5451

Now that the tokenizer is ready, the data needs to be tokenized, and sequences
padded to a maximum length. Since we are interested in comparing performance
with the previosly trained model,we can use the same setting of sampling a maximum of 150 words of the review.

In [12]:
# transformation functions to be used with the dataset
def encode_pad_transform(sample):
  encoded = imdb_encoder.encode(sample.numpy())
  pad = sequence.pad_sequences([encoded], padding="post", maxlen=150)

  return np.array(pad[0], dtype=np.int64)

def encode_tf_fn(sample, label):
  encoded = tf.py_function(encode_pad_transform, inp=[sample], Tout=(tf.int64))
  encoded.set_shape([None])
  label.set_shape([])

  return encoded, label

In [13]:
# test the transformation on a small subset
subset = imdb_train.take(10)
tst = subset.map(encode_tf_fn)

In [14]:
for review, label in tst.take(1):
  print(review, label)
  print("\n", imdb_encoder.decode(review))

tf.Tensor(
[89793 90851 41810 64523 42842 46066 68939 53750 64766 77911 56622 65755
 81772 46086 74649 85007 75996 50792 86109 92051 63423 81211 89793 59535
 59974 64766 83530 58701 88357 56622 72891 63079 83530 92051 65894 54510
 73951 88272 89793 46066 68016 80829 14341 89793 46066 53613 41810 81948
 45799 40028 35243 77544 87326 74511 78243 62805 86117 92855 89120 87326
 27078 66602 86117 81259 83530 28238 76444 66374 25422 30191 59010 79060
 85797 55547 91973 43674 66367 44793 49356 46086 90851 67886 81211 87681
 78243 59698 25928 56622 87681 46066 52058 90851 43014 56722 83936 74977
 91156 50917 82235 81051 52058 91353 86109 14776 89162 89793 79161 77699
 68016 89162 81772 46086 68016 90571 91128 50917 54510 35098 40967 74882
 48263     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0], shape=(150,), dtype=int64) tf.Tensor(0, shape=(), dtype=int64)

 this was 

Finally, the data is encoded using the convenience functions above like so:

In [15]:
# now tokenize/encode/pad all training and testing data
encoded_train = imdb_train.map(encode_tf_fn, num_parallel_calls=tf.data.experimental.AUTOTUNE)
encoded_test = imdb_test.map(encode_tf_fn, num_parallel_calls=tf.data.experimental.AUTOTUNE)

At this point, all the training and test data is ready for training.

## Loading pre-trained GloVe embeddings

The next step is the foremost step in transfer learning – loading the pre-trained GloVe embeddings and using these as the weights of the embedding layer.